In [20]:
import pandas as pd
from sklearn.decomposition import IncrementalPCA, PCA

from lung_cancer.connection_settings import get_connection_string, TABLE_LABELS, TABLE_FEATURES, TABLE_PCA_FEATURES, IMAGES_FOLDER
from lung_cancer.connection_settings import TABLE_PATIENTS, TABLE_TRAIN_ID, MICROSOFTML_MODEL_NAME, TABLE_PREDICTIONS, FASTTREE_MODEL_NAME, TABLE_CLASSIFIERS
from lung_cancer.lung_cancer_utils import compute_features, train_test_split, average_pool, gather_image_paths, insert_model, create_formula, roc

from revoscalepy import rx_import, RxSqlServerData, rx_data_step, RxInSqlServer, RxLocalSeq, rx_set_compute_context
from revoscalepy import RxSqlServerData, RxInSqlServer, RxLocalSeq, rx_set_compute_context, rx_data_step
from microsoftml import rx_fast_trees
from microsoftml import rx_predict as ml_predict

In [21]:
connection_string = get_connection_string()
sql = RxInSqlServer(connection_string=connection_string)
local = RxLocalSeq()
rx_set_compute_context(local)

DRIVER={ODBC Driver 13 for SQL Server};SERVER=TYLER-LAPTOP\TYLERSQLSERVER;PORT=21816;DATABASE=lung_cancer_database;UID=demo;PWD=D@tascience


In [ ]:
print("Gathering patients and labels")
query = "SELECT patient_id, label FROM {}".format(TABLE_LABELS)
data_sql = RxSqlServerData(sql_query=query, connection_string=connection_string)
data = rx_import(data_sql)

data["label"] = data["label"].astype(bool)
n_patients = 200    # How many patients do we featurize images for?
data = data.head(n_patients)
print(data.head())

Gathering patients and labels
Rows Read: 1393, Total Rows Processed: 1393, Total Chunk Time: 0.008 seconds 
                         patient_id  label
0  0015ceb851d7251b8f399e39779d1e7d   True
1  0030a160d58723ff36d73f41b170ec21  False
2  003f41c78e6acfa92430a057ac0b306e  False
3  006b96310a37b36cccb2ab48d10b49a3   True
4  008464bb8521d09a42985dd8add3d0d2   True


In [ ]:
data_to_featurize = gather_image_paths(data, IMAGES_FOLDER)
print(data_to_featurize.head())

Gathered 195 images for patient #0 with id: 0015ceb851d7251b8f399e39779d1e7d
Gathered 265 images for patient #1 with id: 0030a160d58723ff36d73f41b170ec21
Gathered 233 images for patient #2 with id: 003f41c78e6acfa92430a057ac0b306e
Gathered 173 images for patient #3 with id: 006b96310a37b36cccb2ab48d10b49a3
Gathered 146 images for patient #4 with id: 008464bb8521d09a42985dd8add3d0d2
Gathered 171 images for patient #5 with id: 0092c13f9e00a3717fdc940641f00015
Gathered 123 images for patient #6 with id: 00986bebc45e12038ef0ce3e9962b51a
Gathered 134 images for patient #7 with id: 00cba091fa4ad62cc3200a657aeb957e
Gathered 135 images for patient #8 with id: 00edff4f51a893d80dae2d42a7f45ad1
Gathered 191 images for patient #9 with id: 0121c2845f2b7df060945b072b2515d7
Gathered 217 images for patient #10 with id: 013395589c01aa01f8df81d80fb0e2b8
Gathered 231 images for patient #11 with id: 01de8323fa065a8963533c4a86f2f6c1
Gathered 159 images for patient #12 with id: 01e349d34c06410e1da273add27be

In [ ]:
featurized_data = compute_features(data_to_featurize, MICROSOFTML_MODEL_NAME, compute_context=sql)
print(featurized_data.head())

In [ ]:
pooled_data = average_pool(data, featurized_data)
print(pooled_data)
features_sql = RxSqlServerData(table=TABLE_FEATURES, connection_string=connection_string)
rx_data_step(input_data=pooled_data, output_file=features_sql, overwrite=True)

In [ ]:
resample = False
if resample:
    print("Performing Train Test Split")
    p = 80
    train_test_split(TABLE_TRAIN_ID, TABLE_PATIENTS, p, connection_string=connection_string)

In [ ]:
n = min(485, n_patients)    # 485 features is the most that can be handled right now
#pca = IncrementalPCA(n_components=n, whiten=True, batch_size=100)
pca = PCA(n_components=n, whiten=True)

def apply_pca(dataset, context):
    dataset = pd.DataFrame(dataset)
    feats = dataset.drop(["label", "patient_id"], axis=1)
    feats = pca.transform(feats)
    feats = pd.DataFrame(data=feats, index=dataset.index.values, columns=["pc" + str(i) for i in range(feats.shape[1])])
    dataset = pd.concat([dataset[["label", "patient_id"]], feats], axis=1)
    return dataset

query = "SELECT * FROM {} WHERE patient_id IN (SELECT patient_id FROM {})".format(TABLE_FEATURES, TABLE_TRAIN_ID)
train_data_sql = RxSqlServerData(sql_query=query, connection_string=connection_string)
train_data = rx_import(input_data=train_data_sql)
train_data = train_data.drop(["label", "patient_id"], axis=1)
pca.fit(train_data)

rx_set_compute_context(local)
pca_features_sql = RxSqlServerData(table=TABLE_PCA_FEATURES, connection_string=connection_string)
rx_data_step(input_data=features_sql, output_file=pca_features_sql, overwrite=True, transform_function=apply_pca)

In [ ]:
# Point to the SQL table with the training data
column_info = {'label': {'type': 'integer'}}
query = "SELECT * FROM {} WHERE patient_id IN (SELECT patient_id FROM {})".format(TABLE_PCA_FEATURES, TABLE_TRAIN_ID)
print(query)
#train_sql = RxSqlServerData(sql_query=query, connection_string=connection_string, column_info=column_info)
train_sql = RxSqlServerData(sql_query=query, connection_string=connection_string)

In [ ]:
formula = create_formula(train_sql)
print("Formula:", formula)

In [ ]:
# Fit a classification model
classifier = rx_fast_trees(formula=formula,
                           data=train_sql,
                           num_trees=500,
                           method="binary",
                           random_seed=5,
                           compute_context=sql)
print(classifier)

In [ ]:
# Serialize LGBMRegressor model and insert into table
insert_model(TABLE_CLASSIFIERS, connection_string, classifier, FASTTREE_MODEL_NAME)   # TODO: Do table insertions in sql

In [ ]:
# Point to the SQL table with the testing data
query = "SELECT * FROM {} WHERE patient_id NOT IN (SELECT patient_id FROM {})".format(TABLE_PCA_FEATURES, TABLE_TRAIN_ID)
print(query)
test_sql = RxSqlServerData(sql_query=query, connection_string=connection_string)#, column_info=column_info

In [ ]:
# Make predictions on the test data
predictions = ml_predict(classifier, data=test_sql, extra_vars_to_write=["label", "patient_id"])
print(predictions.head())

In [ ]:
predictions_sql = RxSqlServerData(table=TABLE_PREDICTIONS, connection_string=connection_string)
rx_data_step(predictions, predictions_sql, overwrite=True)

In [ ]:
# Evaluate model using ROC
roc(predictions["label"], predictions["Probability"])

In [ ]:
# Specify patient to make prediction for
PatientIndex = 9

In [ ]:
# Select patient data
query = "SELECT TOP(1) * FROM {} AS t1 INNER JOIN {} AS t2 ON t1.patient_id = t2.patient_id WHERE t2.idx = {}".format(TABLE_PCA_FEATURES, TABLE_PATIENTS, PatientIndex)
print(query)
patient_sql = RxSqlServerData(sql_query=query, connection_string=connection_string)

In [ ]:
# Make Prediction on a single patient
predictions = ml_predict(classifier, data=patient_sql, extra_vars_to_write=["label", "patient_id"])


print("The probability of cancer for patient {} with patient_id {} is {}%".format(PatientIndex, predictions["patient_id"].iloc[0], predictions["Probability"].iloc[0]*100))
if predictions["label"].iloc[0] == 0:
    print("Ground Truth: This patient does not have cancer")
else:
    print("Ground Truth: This patient does have cancer")